In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# ================================
# 1. Load and Format Each Dataset
# ================================
def load_macro_csv(filepath, col_name):
    df = pd.read_csv(filepath)
    df.columns = ['date', col_name]
    df['date'] = pd.to_datetime(df['date'])
    return df

# Change base path according to your environment

base_path = Path("/Users/aliyaospanova/Desktop/stock_prediction_project")
raw_data_dir = base_path / "data" / "raw"
processed_data_dir = base_path / "data" / "processed"
macro_data_dir = base_path / "data" / "raw" / "macroeconomics"
results_dir = base_path / "results"
plots_dir = results_dir / "plots"

cpi = load_macro_csv(macro_data_dir / "CPIAUCSL.csv", "cpi")
fed_rate = load_macro_csv(macro_data_dir / "FEDFUNDS.csv", "fed_rate")
unemployment_rate = load_macro_csv(macro_data_dir / "UNRATE.csv", "unemployment_rate")
# retail_sales = load_macro_csv(macro_data_dir / "RSXFS.csv", "retail_sales")  # dropped due to high correlation with cpi
# # close is also dropped due to correlation with cpi = 0.95
industrial_output = load_macro_csv(macro_data_dir / "INDPRO.csv", "industrial_output")
oil_price = load_macro_csv(macro_data_dir / "DCOILWTICO.csv", "oil_price")



In [25]:
# ================================
# 2. Load and Process GSPC Returns
# ================================
sp500 = pd.read_csv(macro_data_dir / "GSPC_historical_data.csv")
sp500 = sp500[['Date', 'Close']].copy()
sp500.columns = ['date', 'close']
sp500['date'] = pd.to_datetime(sp500['date'])
sp500 = sp500.sort_values('date')
sp500['sp500_return'] = sp500['close'].pct_change()
sp500 = sp500.dropna(subset=['sp500_return'])

print(sp500.head())

# ================================
# 3. Forward-Fill Monthly Data to Daily
# ================================
def monthly_to_daily(df):
    return df.set_index('date').resample('D').ffill().reset_index()

macro_dfs = [
    monthly_to_daily(df) for df in [cpi, fed_rate, unemployment_rate, industrial_output, oil_price]
]

# Add sp500 (already daily)
macro_dfs.append(sp500)

print(macro_dfs)

# ================================
# Align All Data to Common Daily Range
# ================================
min_date = max(df['date'].min() for df in macro_dfs)
max_date = min(df['date'].max() for df in macro_dfs)
common_dates = pd.date_range(min_date, max_date, freq='D')

def align_to_common_dates(df):
    df = df.set_index('date').reindex(common_dates).ffill().reset_index()
    df.columns = ['date'] + list(df.columns[1:])
    return df

macro_dfs = [align_to_common_dates(df) for df in macro_dfs]

        date        close  sp500_return
1 2010-01-06  1137.140015      0.000546
2 2010-01-07  1141.689941      0.004001
3 2010-01-08  1144.979980      0.002882
4 2010-01-11  1146.979980      0.001747
5 2010-01-12  1136.219971     -0.009381
[           date      cpi
0    2010-01-01  217.488
1    2010-01-02  217.488
2    2010-01-03  217.488
3    2010-01-04  217.488
4    2010-01-05  217.488
...         ...      ...
4713 2022-11-27  298.708
4714 2022-11-28  298.708
4715 2022-11-29  298.708
4716 2022-11-30  298.708
4717 2022-12-01  298.808

[4718 rows x 2 columns],            date  fed_rate
0    2010-01-01      0.11
1    2010-01-02      0.11
2    2010-01-03      0.11
3    2010-01-04      0.11
4    2010-01-05      0.11
...         ...       ...
4713 2022-11-27      3.78
4714 2022-11-28      3.78
4715 2022-11-29      3.78
4716 2022-11-30      3.78
4717 2022-12-01      4.10

[4718 rows x 2 columns],            date  unemployment_rate
0    2010-01-01                9.8
1    2010-01-02          

In [26]:
# ================================
# 4. Merge All on Date
# ================================
macro_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), macro_dfs)
macro_df = macro_df.sort_values('date').ffill().dropna().reset_index(drop=True)

print(macro_df)

           date      cpi  fed_rate  unemployment_rate  industrial_output  \
0    2010-01-06  217.488      0.11                9.8            89.1897   
1    2010-01-07  217.488      0.11                9.8            89.1897   
2    2010-01-08  217.488      0.11                9.8            89.1897   
3    2010-01-09  217.488      0.11                9.8            89.1897   
4    2010-01-10  217.488      0.11                9.8            89.1897   
...         ...      ...       ...                ...                ...   
4708 2022-11-27  298.708      3.78                3.6           103.1058   
4709 2022-11-28  298.708      3.78                3.6           103.1058   
4710 2022-11-29  298.708      3.78                3.6           103.1058   
4711 2022-11-30  298.708      3.78                3.6           103.1058   
4712 2022-12-01  298.808      4.10                3.5           101.8266   

      oil_price        close  sp500_return  
0         83.12  1137.140015      0.000546

In [27]:
# ================================
# 5. Handle Missing Values Before Normalization
# ================================
macro_df = macro_df.dropna(subset=['cpi', 'fed_rate', 'unemployment_rate', 'industrial_output', 'oil_price', 'sp500_return', 'close'])
macro_df.to_csv( processed_data_dir / "macro_data_cleaned.csv" , index=False)
print(macro_df)

           date      cpi  fed_rate  unemployment_rate  industrial_output  \
0    2010-01-06  217.488      0.11                9.8            89.1897   
1    2010-01-07  217.488      0.11                9.8            89.1897   
2    2010-01-08  217.488      0.11                9.8            89.1897   
3    2010-01-09  217.488      0.11                9.8            89.1897   
4    2010-01-10  217.488      0.11                9.8            89.1897   
...         ...      ...       ...                ...                ...   
4708 2022-11-27  298.708      3.78                3.6           103.1058   
4709 2022-11-28  298.708      3.78                3.6           103.1058   
4710 2022-11-29  298.708      3.78                3.6           103.1058   
4711 2022-11-30  298.708      3.78                3.6           103.1058   
4712 2022-12-01  298.808      4.10                3.5           101.8266   

      oil_price        close  sp500_return  
0         83.12  1137.140015      0.000546

In [28]:
# ================================
# 8. Visualize Correlation Heatmap
# ================================
plt.figure(figsize=(10, 8))
sns.heatmap(macro_df[features].corr(), annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation Heatmap of Normalized Macroeconomic Features")
plt.tight_layout()
plt.savefig(plots_dir/ "macro_feature_correlation_heatmap.png")
plt.show()

KeyError: "['retail_sales'] not in index"

<Figure size 1000x800 with 0 Axes>

In [21]:
# ================================
# 9. Extract Top 5 Correlated Feature Pairs
# ================================
corr_matrix = macro_df[features].corr().abs()
mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
corr_pairs = corr_matrix.where(mask).stack().reset_index()
corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
top_corr = corr_pairs.sort_values(by='Correlation', ascending=False).head(5)
top_corr.to_csv(processed_data_dir / "top_correlated_feature_pairs.csv", index=False)
print("Top correlated feature pairs saved to top_correlated_feature_pairs.csv")

"""
Feature 1	Feature 2	Correlation
cpi	retail_sales	0.9842358418480940
unemployment_rate	industrial_output	0.8811151146734950
fed_rate	unemployment_rate	0.6168739438031150
unemployment_rate	retail_sales	0.5945983133613900
cpi	unemployment_rate	0.5942154359701660
"""

Top correlated feature pairs saved to top_correlated_feature_pairs.csv


'\nFeature 1\tFeature 2\tCorrelation\ncpi\tretail_sales\t0.9842358418480940\nunemployment_rate\tindustrial_output\t0.8811151146734950\nfed_rate\tunemployment_rate\t0.6168739438031150\nunemployment_rate\tretail_sales\t0.5945983133613900\ncpi\tunemployment_rate\t0.5942154359701660\n'

In [22]:
# ================================
# 10. Optional Redundant Feature Filtering (Threshold > 0.95)
# ================================
high_corr_pairs = corr_pairs[corr_pairs['Correlation'] > 0.95]
redundant_features = set(high_corr_pairs['Feature 2'])
print("Redundant features to consider removing:", redundant_features)

Redundant features to consider removing: {'close', 'retail_sales'}
